In [9]:
from pyscf import gto, scf
from pyscf.geomopt.geometric_solver import optimize
import dftd4.pyscf as disp

# Function to perform DFT calculation with or without dispersion
def perform_SCF_calculation(mol, input_xyz, method=None, dispersion=None):   
    mol.atom = input_xyz
    mol.build()
    if method:
        mf =scf.RHF(mol)
    else:
        mf = scf.RKS(mol)
        mf.xc ='B3LYP'
 
    mf = mf.density_fit(auxbasis='def2-universal-jkfit') 
    
    if dispersion:
        mf = disp.energy(mf).run()
    else:
        mf = mf.run()
    
    return mf


[taamangtchu-Aspire-A515-45:03001] mca_base_component_repository_open: unable to open mca_btl_openib: librdmacm.so.1: cannot open shared object file: No such file or directory (ignored)
/home/taamangtchu/mambaforge/envs/qiskit-env/lib/python3.10/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [10]:
def min_TRIC_energy(logfile):
    """Read the geometry optimization logfile and extract
      the last "converged SCF energy", that is the minimum 
      energy of the state, at the end that file
    """

    with open(logfile, "r") as f:
        log_contents = f.read()

    converged_scf_energy = None
    lines = log_contents.split("\n")
    for line in reversed(lines):
        if "converged SCF energy" in line:
            converged_scf_energy = line.split("=")[-1].strip()
            break
    
    return float(converged_scf_energy)


In [11]:
# Define the molecule once
# mol_xyz = """
#     O  -1.65542061  -0.12330038   0.00000000
#     O   1.24621244   0.10268870   0.00000000
#     H  -0.70409026   0.03193167   0.00000000
#     H  -2.03867273   0.75372294   0.00000000
#     H   1.57598558  -0.38252146  -0.75856129
#     H   1.57598558  -0.38252146   0.75856129
#     """
mol_xyz = """O        0.0000   0.0000   0.1173
             H        0.0000   0.7572	 -0.4692
             H        0.0000  -0.7572	 -0.4692"""
mol = gto.M(
    basis =  "6-31G",
    output = "geom_log.txt"
)

overwrite output file: geom_log.txt


In [12]:
# Without dispersion
mf_d4_disp = perform_SCF_calculation(mol, mol_xyz, method='HF', dispersion = 'd4')
mol_TRIC = optimize(mf_d4_disp, verbose = 4)

geometric-optimize called with the following command line:
/home/taamangtchu/mambaforge/envs/qiskit-env/lib/python3.10/site-packages/ipykernel_launcher.py -f /home/taamangtchu/.local/share/jupyter/runtime/kernel-v2-1961dmwjMUdOBbyl.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  **

In [13]:
print(mol_TRIC.tofile('opt_d4_disp.xyz'),'\n')

e_d4_disp = min_TRIC_energy("geom_log.txt")
print(min_TRIC_energy("geom_log.txt"))

3
XYZ from PySCF
O           0.00000       -0.00000        0.08260
H           0.00000        0.78517       -0.45185
H          -0.00000       -0.78517       -0.45185 



In [16]:
mol_xyz_d4 = open('opt_d4_disp.xyz').read()
mol_xyz_d4 = '\n'.join(mol_xyz_d4.strip().split('\n')([0]+[2:])
print(mol_xyz_d4)

SyntaxError: invalid syntax (3187841203.py, line 2)

In [14]:
mdft_d4_disp = perform_SCF_calculation(mol, mol_xyz, method='DFT', dispersion = 'd4')
ee = mdft_d4_disp.e_tot


In [13]:
from pyscf import fci

myfci = fci.FCI(mf_d4_disp)
myfci.kernel()

(-76.12183995103189,
 FCIvector([[ 9.77293628e-01,  5.34055192e-18, -9.36420481e-03, ...,
             -3.20235236e-22, -5.36935813e-22, -2.78649990e-07],
            [ 5.34055192e-18, -1.36461416e-02,  1.29758392e-17, ...,
              1.28570146e-21,  1.70023335e-23,  4.52184319e-22],
            [-9.36420481e-03,  1.29758392e-17, -2.20702000e-02, ...,
              1.11208442e-22, -6.52120405e-24, -3.49676941e-07],
            ...,
            [-3.20235236e-22,  1.28570146e-21,  1.11208442e-22, ...,
             -8.56723415e-10,  7.58104574e-10,  9.81033165e-24],
            [-5.36935813e-22,  1.70023335e-23, -6.52120405e-24, ...,
              7.58104574e-10, -1.70409488e-09, -2.40998710e-24],
            [-2.78649990e-07,  4.52184319e-22, -3.49676941e-07, ...,
              9.81033165e-24, -2.40998710e-24, -1.72239345e-08]]))

In [14]:
# print('Without d4 dispersion', mf_no_disp.e_tot, 'Delta FCI', abs(mf_no_disp.e_tot-myfci.e_tot)*627.5)
print(f'd4 disp min energy = {e_d4_disp} a.u. \n Delta FCI = {abs(e_d4_disp-myfci.e_tot)*627.5} kcal/cal')
print(f'd4 disp energy = {mf_d4_disp.e_tot} a.u. \n Delta FCI = {abs(mf_d4_disp.e_tot-myfci.e_tot)*627.5} kcal/cal')


d4 disp min energy = -75.9863152288775 a.u. 
 Delta FCI = 85.04176315187738 kcal/cal
d4 disp energy = -75.98494487246366 a.u. 
 Delta FCI = 85.90166180156442 kcal/cal


In [6]:
from rdkit import Chem
from rdkit.Chem import RDConfig
import sys
from pathlib import Path

sa_score_dir = Path(RDConfig.RDContribDir) / 'SA_Score'
sys.path.append(str(sa_score_dir))
import sascorer
mol = Chem.MolFromSmiles('c1ccccc1') 

# Calculate SAScore
sascorer.calculateScore(mol)



1.0

In [7]:
from rdkit import Chem
from rdkit.Chem import RDConfig
import sys
from pathlib import Path

sa_score_dir = Path(RDConfig.RDContribDir) / 'SA_Score'
sys.path.append(sa_score_dir)
import sascorer
mol = Chem.MolFromSmiles('c1ccccc1') 

# Calculate SAScore
sascorer.calculateScore(mol)



1.0

In [8]:
from rdkit import Chem
from rdkit.Chem import RDConfig
import sys
from pathlib import Path

sys.path.append(Path(RDConfig.RDContribDir) / 'SA_Score')
import sascorer
mol = Chem.MolFromSmiles('c1ccccc1') 

# Calculate SAScore
sascorer.calculateScore(mol)



1.0